# 작곡: 음악을 생성하는 모델을 훈련하기

In [1]:
import os
import pickle
import numpy
from music21 import note, chord

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.utils import plot_model

from models.RNNAttention import get_distinct, create_lookups, prepare_sequences, get_music_list, create_network

Using TensorFlow backend.
/home/haesun/github/GDL_code/env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/haesun/github/GDL_code/env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/haesun/github/GDL_code/env/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/haesun/github/GDL_co

## 파라미터 설정

In [2]:
# 실행 파라미터
section = 'compose'
run_id = '0006'
music_name = 'cello'

run_folder = 'run/{}/'.format(section)
run_folder += '_'.join([run_id, music_name])


store_folder = os.path.join(run_folder, 'store')
data_folder = os.path.join('data', music_name)

if not os.path.exists(run_folder):
    os.mkdir(run_folder)
    os.mkdir(os.path.join(run_folder, 'store'))
    os.mkdir(os.path.join(run_folder, 'output'))
    os.mkdir(os.path.join(run_folder, 'weights'))
    os.mkdir(os.path.join(run_folder, 'viz'))
    


mode = 'build' # 'load' # 

# 데이터 파라미터
intervals = range(1)
seq_len = 32

# 하이퍼퍼라미터
embed_size = 100
rnn_units = 256
use_attention = True

## 악보 추출

In [3]:
if mode == 'build':
    
    music_list, parser = get_music_list(data_folder)
    print(len(music_list), 'files in total')

    notes = []
    durations = []

    for i, file in enumerate(music_list):
        print(i+1, "Parsing %s" % file)
        original_score = parser.parse(file).chordify()
        

        for interval in intervals:

            score = original_score.transpose(interval)

            notes.extend(['START'] * seq_len)
            durations.extend([0]* seq_len)

            for element in score.flat:
                
                if isinstance(element, note.Note):
                    if element.isRest:
                        notes.append(str(element.name))
                        durations.append(element.duration.quarterLength)
                    else:
                        notes.append(str(element.nameWithOctave))
                        durations.append(element.duration.quarterLength)

                if isinstance(element, chord.Chord):
                    notes.append('.'.join(n.nameWithOctave for n in element.pitches))
                    durations.append(element.duration.quarterLength)

    with open(os.path.join(store_folder, 'notes'), 'wb') as f:
        pickle.dump(notes, f) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(store_folder, 'durations'), 'wb') as f:
        pickle.dump(durations, f) 
else:
    with open(os.path.join(store_folder, 'notes'), 'rb') as f:
        notes = pickle.load(f) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(store_folder, 'durations'), 'rb') as f:
        durations = pickle.load(f) 

1 files in total
1 Parsing data/cello/cs1-2all.mid


## 룩업 테이블 만들기

In [4]:
# 고유한 음표와 박자 얻어오기
note_names, n_notes = get_distinct(notes)
duration_names, n_durations = get_distinct(durations)
distincts = [note_names, n_notes, duration_names, n_durations]

with open(os.path.join(store_folder, 'distincts'), 'wb') as f:
    pickle.dump(distincts, f)

# 음표와 박자 룩업 딕셔너리 만들고 저장하기
note_to_int, int_to_note = create_lookups(note_names)
duration_to_int, int_to_duration = create_lookups(duration_names)
lookups = [note_to_int, int_to_note, duration_to_int, int_to_duration]

with open(os.path.join(store_folder, 'lookups'), 'wb') as f:
    pickle.dump(lookups, f)

In [5]:
print('\nnote_to_int')
note_to_int


note_to_int


{'A2': 0,
 'A2.E3.A3': 1,
 'A2.E3.C4': 2,
 'A3': 3,
 'B-3': 4,
 'B2': 5,
 'B3': 6,
 'C#3': 7,
 'C#4': 8,
 'C3': 9,
 'C4': 10,
 'D2': 11,
 'D3': 12,
 'D3.A3': 13,
 'D4': 14,
 'E-3': 15,
 'E2': 16,
 'E3': 17,
 'E3.A3': 18,
 'E3.B3': 19,
 'E4': 20,
 'F#2': 21,
 'F#3': 22,
 'F#4': 23,
 'F3': 24,
 'F4': 25,
 'G#2': 26,
 'G#3': 27,
 'G2': 28,
 'G2.D3.B3': 29,
 'G3': 30,
 'G4': 31,
 'START': 32}

In [6]:
print('\nduration_to_int')
duration_to_int


duration_to_int


{0: 0, 0.25: 1, 0.5: 2, Fraction(2, 3): 3, 0.75: 4, 1.0: 5, 1.25: 6}

## 신경망에 사용할 시퀀스 준비하기

In [7]:
network_input, network_output = prepare_sequences(notes, durations, lookups, distincts, seq_len)

In [8]:
print('pitch input')
print(network_input[0][0])
print('duration input')
print(network_input[1][0])
print('pitch output')
print(network_output[0][0])
print('duration output')
print(network_output[1][0])

pitch input
[32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32 32
 32 32 32 32 32 32 32 32]
duration input
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
pitch output
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0.]
duration output
[0. 1. 0. 0. 0. 0. 0.]


## 신경망 만들기

In [9]:
model, att_model = create_network(n_notes, n_durations, embed_size, rnn_units, use_attention)
model.summary()

W0923 21:22:35.540700 140547797157696 deprecation_wrapper.py:119] From /home/haesun/github/GDL_code/env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0923 21:22:35.555252 140547797157696 deprecation_wrapper.py:119] From /home/haesun/github/GDL_code/env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0923 21:22:35.572699 140547797157696 deprecation_wrapper.py:119] From /home/haesun/github/GDL_code/env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0923 21:22:35.983621 140547797157696 deprecation_wrapper.py:119] From /home/haesun/github/GDL_code/env/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.comp

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 100)    3300        input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, None, 100)    700         input_2[0][0]                    
__________________________________________________________________________________________________
concatenat

In [10]:
plot_model(model, to_file=os.path.join(run_folder ,'viz/model.png'), show_shapes = True, show_layer_names = True)

## 신경망 훈련하기

In [11]:
weights_folder = os.path.join(run_folder, 'weights')
# model.load_weights(os.path.join(weights_folder, "weights.h5"))

In [12]:
weights_folder = os.path.join(run_folder, 'weights')

checkpoint1 = ModelCheckpoint(
    os.path.join(weights_folder, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

checkpoint2 = ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 10
)


callbacks_list = [
    checkpoint1
    , checkpoint2
    , early_stopping
 ]

model.save_weights(os.path.join(weights_folder, "weights.h5"))
model.fit(network_input, network_output
          , epochs=2000000, batch_size=32
          , validation_split = 0.2
          , callbacks=callbacks_list
          , shuffle=True
         )



W0923 21:22:36.149396 140547797157696 deprecation_wrapper.py:119] From /home/haesun/github/GDL_code/env/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0923 21:22:36.854011 140547797157696 deprecation.py:323] From /home/haesun/github/GDL_code/env/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 732 samples, validate on 184 samples
Epoch 1/2000000
732/732 [==============================] - 2s 3ms/step - loss: 3.4665 - pitch_loss: 2.9775 - duration_loss: 0.4890 - val_loss: 3.3925 - val_pitch_loss: 2.9883 - val_duration_loss: 0.4042
Epoch 2/2000000
732/732 [==============================] - 1s 2ms/step - loss: 3.1528 - pitch_loss: 2.8316 - duration_loss: 0.3212 - val_loss: 3.3703 - val_pitch_loss: 2.9626 - val_duration_loss: 0.4077
Epoch 3/2000000
732/732 [==============================] - 1s 2ms/step - loss: 3.1157 - pitch_loss: 2.8066 - duration_loss: 0.3092 - val_loss: 3.3388 - val_pitch_loss: 2.9333 - val_duration_loss: 0.4055
Epoch 4/2000000
732/732 [==============================] - 1s 2ms/step - loss: 3.0918 - pitch_loss: 2.7886 - duration_loss: 0.3032 - val_loss: 3.4365 - val_pitch_loss: 3.0038 - val_duration_loss: 0.4327
Epoch 5/2000000
732/732 [==============================] - 1s 2ms/step - loss: 3.0919 - pitch_loss: 2.7813 - duration_loss: 0.3106 - val_loss:

Epoch 41/2000000
732/732 [==============================] - 1s 2ms/step - loss: 1.3768 - pitch_loss: 1.2810 - duration_loss: 0.0958 - val_loss: 1.5423 - val_pitch_loss: 1.4084 - val_duration_loss: 0.1340
Epoch 42/2000000
732/732 [==============================] - 1s 2ms/step - loss: 1.3611 - pitch_loss: 1.2687 - duration_loss: 0.0924 - val_loss: 1.5452 - val_pitch_loss: 1.4183 - val_duration_loss: 0.1269
Epoch 43/2000000
732/732 [==============================] - 1s 2ms/step - loss: 1.3192 - pitch_loss: 1.2306 - duration_loss: 0.0886 - val_loss: 1.5118 - val_pitch_loss: 1.3747 - val_duration_loss: 0.1371
Epoch 44/2000000
732/732 [==============================] - 1s 2ms/step - loss: 1.2530 - pitch_loss: 1.1809 - duration_loss: 0.0722 - val_loss: 1.4400 - val_pitch_loss: 1.3191 - val_duration_loss: 0.1209
Epoch 45/2000000
732/732 [==============================] - 1s 2ms/step - loss: 1.2146 - pitch_loss: 1.1366 - duration_loss: 0.0780 - val_loss: 1.3839 - val_pitch_loss: 1.2626 - val_du

732/732 [==============================] - 1s 2ms/step - loss: 0.2080 - pitch_loss: 0.2001 - duration_loss: 0.0079 - val_loss: 0.0590 - val_pitch_loss: 0.0584 - val_duration_loss: 5.4350e-04
Epoch 122/2000000
732/732 [==============================] - 1s 2ms/step - loss: 0.1543 - pitch_loss: 0.1464 - duration_loss: 0.0079 - val_loss: 0.1057 - val_pitch_loss: 0.0868 - val_duration_loss: 0.0190
Epoch 123/2000000
732/732 [==============================] - 1s 2ms/step - loss: 0.1535 - pitch_loss: 0.1390 - duration_loss: 0.0145 - val_loss: 0.1207 - val_pitch_loss: 0.1141 - val_duration_loss: 0.0066
Epoch 124/2000000
732/732 [==============================] - 1s 2ms/step - loss: 0.1620 - pitch_loss: 0.1361 - duration_loss: 0.0259 - val_loss: 0.0109 - val_pitch_loss: 0.0100 - val_duration_loss: 8.8783e-04
